In [4]:
import torch
import pytorch_lightning as pl
from network import model,model2
from audtorch.metrics import PearsonR
from eriksdataset import EEG_Dataset
from torchmetrics.wrappers import Running
from torchmetrics.aggregation import MeanMetric
from torch.utils.data import DataLoader,TensorDataset
from pytorch_lightning.callbacks import EarlyStopping,ModelCheckpoint,ModelSummary
torch.set_float32_matmul_precision('medium')
import os
from diffnet import EEG_Conformer


In [5]:
def findfiles(search_dir, prefix):
    matching_files = []

    for dirpath, _, filenames in os.walk(search_dir):
        for filename in filenames:
            if filename.startswith(prefix):
                full_path = os.path.join(dirpath, filename)
                matching_files.append(full_path)

    return matching_files

In [6]:
class Pearsonr(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self,P,O):

        (n, t) = O.shape      # n traces of t samples
        (n_bis, m) = P.shape  # n predictions for each of m candidates

        DO = O - (torch.einsum("nt->t", O) / n) # compute O - mean(O)
        DP = P - (torch.einsum("nm->m", P) / n) # compute P - mean(P)

        cov = torch.einsum("nm,nt->mt", DP, DO)

        varP = torch.einsum("nm,nm->m", DP, DP)
        varO = torch.einsum("nt,nt->t", DO, DO)
        tmp = torch.einsum("m,t->mt", varP, varO)

        return cov / torch.sqrt(tmp)

In [7]:
class CLoss(torch.nn.Module):
    def __init__(self,mean,lin,scale,temp):
        super().__init__()
        self.mean = mean
        self.lin = lin
        self.scale = scale
        self.pear = Pearsonr()
        self.temp = temp
    
    def forward(self,eemb,aamb):
        if self.mean:
            aamb = torch.mean(aamb,dim=1,keepdim=True)
            eemb = torch.mean(eemb,dim=1,keepdim=True)


        if self.scale:
            eemb =(eemb - eemb.mean(dim=1,keepdim=True))/eemb.std(dim=1,keepdim=True)
            aamb =(aamb - aamb.mean(dim=1,keepdim=True))/aamb.std(dim=1,keepdim=True)

        
        #eemb = eemb/eemb.norm(2,dim=1,keepdim=True)
        #aamb = aamb/aamb.norm(2,dim=1,keepdim=True)


        

        logits = (self.pear(eemb,aamb))*torch.exp(self.temp)


        t= torch.arange(len(eemb[0]),device=eemb.device)


        l1 = torch.nn.functional.cross_entropy(logits,t)
    
        return l1

In [8]:
class Encode(pl.LightningModule):
    def __init__(self):
        super().__init__()
        #self.encoder = model(nblock=2,lin_out=8,kernel_size=3,transformer_hidden=32,n_layers=2,n_heads=2,dff=16,filter_size=16,nconvs=2,times=320)
        #self.encoder = model2(transformer_hidden=32,n_heads=4,dff=128)
        self.encoder = EEG_Conformer()
        self.temp = torch.nn.Parameter(torch.log(torch.ones(1)*5))
        self.loss_fn = CLoss(lin=False,scale=True,mean=False,temp= self.temp)
        self.running_loss = Running(MeanMetric(), window=32)
        self.running_loss2 = Running(MeanMetric(),window=32)
        self.running_loss1 = Running(MeanMetric(), window=32)
        self.running_loss22 = Running(MeanMetric(),window=32)
        self.los = PearsonR()
        #self.accfn = PearsonR(reduction=None)
        self.lr = 3e-5
    

    def forward(self, eeg):
        # in lightning, forward defines the prediction/inference actions
        envelope = self.encoder(eeg)
        return envelope

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop. It is independent of forward
        eeg,att,_ = batch

        #eeg = eeg.unsqueeze(1)

        eeg_emb = self(eeg)

        #loss1 = self.loss_fn(eeg_emb.T,att.T)
        loss2 = -self.los(eeg_emb,att)
        #loss = loss1+loss2


        #self.running_loss(loss1)
        self.running_loss1(loss2)
        #self.log('train_loss', self.running_loss.compute(), on_step=True, prog_bar=True)
        self.log('train_pearson_loss', self.running_loss1.compute(), on_step=True, prog_bar=True)
        #self.log('train_accuracy',)

    


        return loss2

    
    def validation_step(self, batch, batch_idx):
        # training_step defines the train loop. It is independent of forward
        eeg,att,_ = batch

        #eeg = eeg.unsqueeze(1)
        eeg_emb = self(eeg)

        #loss1 = self.loss_fn(eeg_emb.T,att.T)
        loss2 = -self.los(eeg_emb,att)
        #loss = loss1+loss2

        #self.running_loss2(loss1)
        self.running_loss22(loss2)
        #self.log('val_loss', self.running_loss2.compute(), on_step=True, prog_bar=True)
        self.log('pearson_loss', self.running_loss22.compute(), on_step=True, prog_bar=True)

        return loss2

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,800)
        return [optimizer],[scheduler]

In [9]:
from sklearn.model_selection import train_test_split

data = findfiles("EriksholmFiles/","on")
data += findfiles("EriksholmFiles/","off")

trainfiles,valfiles = train_test_split(data,test_size=0.1)

In [10]:
tr_dataset = EEG_Dataset(files=trainfiles,seconds=5,overlap=0.75)
val_dataset = EEG_Dataset(files=valfiles,seconds=5,overlap=0.75)

tr_loader = DataLoader(tr_dataset,batch_size=32,num_workers=3,persistent_workers=True,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=32,num_workers=3,persistent_workers=True)

In [11]:
checkpoint_callback = ModelCheckpoint(monitor='val_loss',dirpath='./EriksholmFiles/',filename='pretrained_weights',auto_insert_metric_name=False)
early_stopping = EarlyStopping('val_loss',patience=10)
ms = ModelSummary(max_depth=3)

In [12]:
enc = Encode()

trainer = pl.Trainer(max_epochs=300,accelerator='gpu',gradient_clip_val=1.0,callbacks=[checkpoint_callback,ms])
trainer.fit(enc, tr_loader,val_dataloaders=val_loader)

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\gauta\anaconda3\envs\keras-jax\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:653: Checkpoint directory C:\Users\gauta\Thesis\PyTorch\EriksholmFiles exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                       | Type            | Params
----------------------------------------------------------------
0  | encoder                    | EEG_Conformer   | 512 K 
1  | encoder.confblock          | ModuleList      | 512 K 
2  | encoder.confblock.0        | Conformer_block | 102 K 
3  | encoder.confblock.1        | Conformer_block | 102 K 
4  | encoder.confblock.2        | Conformer_block | 102 K 
5  | en

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

AssertionError: 

In [ ]:
mod = Encode()

c:\Users\gauta\anaconda3\envs\keras-jax\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [6]:
mod = Encode.load_from_checkpoint("/DTUFiles/pretrained_weights.ckpt")

RuntimeError: Error(s) in loading state_dict for Encode:
	Missing key(s) in state_dict: "encoder.Enc_Pre.CNN.conv_layers.3.weight", "encoder.Enc_Pre.CNN.conv_layers.3.bias", "encoder.Enc_Pre.CNN.conv_layers.4.weight", "encoder.Enc_Pre.CNN.conv_layers.4.bias", "encoder.Enc_Pre.CNN.norm.3.weight", "encoder.Enc_Pre.CNN.norm.3.bias", "encoder.Enc_Pre.CNN.norm.4.weight", "encoder.Enc_Pre.CNN.norm.4.bias", "encoder.Enc_Pre.Convolutions.0.conv_layers.3.weight", "encoder.Enc_Pre.Convolutions.0.conv_layers.3.bias", "encoder.Enc_Pre.Convolutions.0.conv_layers.4.weight", "encoder.Enc_Pre.Convolutions.0.conv_layers.4.bias", "encoder.Enc_Pre.Convolutions.0.norm.3.weight", "encoder.Enc_Pre.Convolutions.0.norm.3.bias", "encoder.Enc_Pre.Convolutions.0.norm.4.weight", "encoder.Enc_Pre.Convolutions.0.norm.4.bias", "encoder.Enc_Pre.Convolutions.1.conv_layers.3.weight", "encoder.Enc_Pre.Convolutions.1.conv_layers.3.bias", "encoder.Enc_Pre.Convolutions.1.conv_layers.4.weight", "encoder.Enc_Pre.Convolutions

In [11]:
from tqdm import tqdm
test_loss = []
test_metric = []
test_masker = []
test_model = enc.cpu().eval()
acc = []

test_metric_dict = {}

with torch.no_grad():
    for batch_data in tqdm(val_loader):
        inputs, outputs,masker = batch_data[0].squeeze(0), batch_data[1].squeeze(0),batch_data[2].squeeze(0)
        #inputs = inputs.unsqueeze(1)

        # Forward pass
        predictions = test_model(inputs)

        # Compute loss
        loss = -PearsonR()(outputs, predictions).mean()

        # Compute metric
        metric = PearsonR(reduction=None)(outputs, predictions)
        metric2 = PearsonR(reduction=None)(masker, predictions)
        accu = torch.sum(metric>metric2)/metric.shape[0]

        test_loss.append(loss.item())
        test_metric.append(metric.mean().detach().cpu().numpy().reshape(-1, 1))
        test_masker.append(metric2.mean().detach().cpu().numpy().reshape(-1, 1))
        acc.append(accu.detach().numpy())
        #test_metric_dict[name] = metric.detach().cpu().numpy().reshape(-1, 1)

100%|██████████| 21/21 [00:21<00:00,  1.01s/it]


In [12]:
import numpy as np
np.mean(acc)

0.62614524

In [15]:
np.mean(acc

[array(0.28125, dtype=float32),
 array(0.40625, dtype=float32),
 array(0.59375, dtype=float32),
 array(0.46875, dtype=float32),
 array(0.6875, dtype=float32),
 array(0.5625, dtype=float32),
 array(0.8125, dtype=float32),
 array(0.8125, dtype=float32),
 array(0.53125, dtype=float32),
 array(0.65625, dtype=float32),
 array(0.59375, dtype=float32),
 array(0.34375, dtype=float32),
 array(0.6875, dtype=float32),
 array(0.625, dtype=float32),
 array(0.59375, dtype=float32),
 array(0.375, dtype=float32),
 array(0.59375, dtype=float32),
 array(0.4375, dtype=float32),
 array(0.5, dtype=float32),
 array(0.40625, dtype=float32),
 array(0.5625, dtype=float32),
 array(0.625, dtype=float32),
 array(0.65625, dtype=float32),
 array(0.53125, dtype=float32),
 array(0.71875, dtype=float32),
 array(0.78125, dtype=float32),
 array(0.40625, dtype=float32),
 array(0.71875, dtype=float32),
 array(0.53125, dtype=float32),
 array(0.46875, dtype=float32),
 array(0.34375, dtype=float32),
 array(0.15625, dtype=flo

In [12]:
np.mean(test_metric)

0.104037605

In [13]:
np.mean(test_masker)

0.032472655

In [18]:
np.mean(test_loss)

-0.1269815012931277

In [2]:
c = model2()
x = torch.rand([1,1,64,256])
out = c(x)
out.shape

torch.Size([1, 256])

In [14]:
c = CNN()
x = torch.rand([1,1,64,256])
out = c(x)
out.shape

torch.Size([1, 20, 32])

In [27]:
class out_model(torch.nn.Module):

    def __init__(self):
        super().__init__()

        self.conv = torch.nn.ConvTranspose1d(32,32,55,stride=9)
        self.conv2 = torch.nn.ConvTranspose1d(32,32,31,1)

    
    def forward(self,x):

        out = self.conv(x)
        out = self.conv2(out)

        return out

In [28]:
c = out_model()
x = torch.rand([1,32,20])
out = c(x)
out.shape

torch.Size([1, 32, 256])

In [23]:
from pytorch_lightning.tuner import Tuner


In [60]:
mod = Encode()
trainer = pl.Trainer()
tuner = Tuner(trainer)

# Run learning rate finder
lr_finder = tuner.lr_find(mod,train_dataloaders=tr_loader,val_dataloaders=val_loader)

# Results can be found in
print(lr_finder.results)

# Plot with
fig = lr_finder.plot(suggest=True)
fig.show()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.2754228703338169
Restoring states from the checkpoint path at c:\Users\gauta\Thesis\PyTorch\.lr_find_4950efaa-0fbd-4ca8-a567-834434e22c27.ckpt


RuntimeError: Can't access the shape of an uninitialized parameter or buffer. This error usually happens in `load_state_dict` when trying to load an uninitialized parameter into an initialized one. Call `forward` to initialize the parameters before accessing their attributes.

In [36]:
print(lr_finder.results)

# Plot with
fig = lr_finder.plot(suggest=True)
fig.show()

NameError: name 'lr_finder' is not defined